<a href="https://colab.research.google.com/github/ldocarvalho/portals-bias/blob/main/portals_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data

In [32]:
import pandas as pd

In [33]:
df_names = ["Source","Title","Content","Bias"]
df_news_aggregator = pd.read_csv('https://raw.githubusercontent.com/ldocarvalho/portals-bias/main/Dataset/News-Aggregator-14/news-aggregator.csv?token=AKX52J7EV5Y2TFAPAPLJ3Q3BYDMV2', names = df_names, skiprows = 1, header = None)
df_news_aggregator

,Source,Title,Content,Bias
0,wsj.com,"EU Week Ahead March 10-14: Bank Resolution, Tr...",The European Union’s to-do list isn’t long thi...,right-center
1,wsj.com,ECB's Noyer not Happy With Euro Strength -- Up...,News Corp is a network of leading companies in...,right-center
2,seattlepi.com,"EBay, Icahn talk smack",Follow @csaid EBay has come out swinging again...,left-center
3,techcrunch.com,EBay Asks Shareholders To Vote Against PayPal ...,The war of words between eBay and its new shar...,left-center
4,valuewalk.com,Carl Icahn: Donahoe Cost eBay Investors More T...,Carl Icahn has again posted an open letter on ...,right-center
...,...,...,...,...
1795,newsone.com,NewsOne Minute: Chris Rock Interviews White Pe...,Chris Rock Interviews White People at a Monste...,left-center
1796,twitchy.com,Chart-topping songwriter Diane Warren: Hope so...,Texas cheerleader and big-game hunter Kendall ...,right
1797,twitchy.com,Reality star Joanna Krupa would love to see ‘t...,Model Joanna Krupa is known for her stints on ...,right
1798,austinchronicle.com,Elaine Stritch (1925-2014),"During her seven decades as a ""Broadway Baby"" ...",left


In [34]:
df_news_july = pd.read_csv('https://raw.githubusercontent.com/ldocarvalho/portals-bias/main/Dataset/News-July-19/news-july.csv?token=AKX52J5KQ3NIC6K7PWM7XPLBYDNKA', names = df_names, skiprows = 1, header = None)
df_news_july

,Source,Title,Content,Bias
0,berkeleyside,"ArchivesJune 10, 2019",When the homeowner did not voluntarily repair ...,left-center
1,berkeleyside,Berkeley homeowner caught in costly code viola...,"Finger-pointing, confusion, worry and mistrust...",left-center
2,berkeleyside,"A small-business success story, Yalis Caf cele...","Yali's Caf 1920 Oxford St. (at Berkeley Way), ...",left-center
3,berkeleyside,Shop Talk: Bluemercury; Airport Home Appliance...,BLUEMERCURY A new business is opening in the o...,left-center
4,berkeleyside,New vision plan imagines 3-mile greenway from ...,Imagine a bike and pedestrian path stretching ...,left-center
...,...,...,...,...
4955,theblaze,Bill Cosbys official Instagram account shares ...,The official Instagram account of actor Bill C...,right
4956,theblaze,WATCH: Students condemn racist remarks they be...,Students at Marymount University in Virginia b...,right
4957,theblaze,Republicans propose constitutional amendment t...,President Donald Trump is supporting a propose...,right
4958,theblaze,108 GOP lawmakers sign letter asking Trump to ...,More than a hundred Republican members of Cong...,right


In [35]:
df_news_february = pd.read_csv('https://raw.githubusercontent.com/ldocarvalho/portals-bias/main/Dataset/News-July-19/news-july.csv?token=AKX52J5KQ3NIC6K7PWM7XPLBYDNKA', names = df_names, skiprows = 1, header = None)
df_news_february

,Source,Title,Content,Bias
0,berkeleyside,"ArchivesJune 10, 2019",When the homeowner did not voluntarily repair ...,left-center
1,berkeleyside,Berkeley homeowner caught in costly code viola...,"Finger-pointing, confusion, worry and mistrust...",left-center
2,berkeleyside,"A small-business success story, Yalis Caf cele...","Yali's Caf 1920 Oxford St. (at Berkeley Way), ...",left-center
3,berkeleyside,Shop Talk: Bluemercury; Airport Home Appliance...,BLUEMERCURY A new business is opening in the o...,left-center
4,berkeleyside,New vision plan imagines 3-mile greenway from ...,Imagine a bike and pedestrian path stretching ...,left-center
...,...,...,...,...
4955,theblaze,Bill Cosbys official Instagram account shares ...,The official Instagram account of actor Bill C...,right
4956,theblaze,WATCH: Students condemn racist remarks they be...,Students at Marymount University in Virginia b...,right
4957,theblaze,Republicans propose constitutional amendment t...,President Donald Trump is supporting a propose...,right
4958,theblaze,108 GOP lawmakers sign letter asking Trump to ...,More than a hundred Republican members of Cong...,right


### Text Pre-processing

In [36]:
def to_lower(text):
  return text.lower()

In [37]:
import re

def remove_numbers(text):
  return re.sub(r'\d+', '', text)

In [38]:
import string
import numpy as np

def remove_puctuation(text):
  symbols = "!\"#$%&()'*+-.,—/:;<=>?@[\]^_`{|}~\n“‘"
  for i in symbols:
    text = text.replace(i, '')
  return text

In [39]:
def preprocess(data):
    data = to_lower(data)
    data = remove_puctuation(data)
    data = remove_numbers(data)
    return data

In [40]:
data = df_news_aggregator.iloc[3]['Content']
data = preprocess(data)
data

'the war of words between ebay and its new shareholder carl icahn continues to rage on today ebay issued a notice of an upcoming annual meeting in which the company asked shareholders flat out to vote against icahn’s proposal to spin off paypal meanwhile icahn issued yet another open letter against current ebay management in which he accused ceo john donahoe of inexcusable incompetence” that cost stockholders  billion carl icahn has also proposed adding two employees from the icahn group to ebay’s board icahn currently owns  of ebay as of february  ebay has also asked shareholders to reject that proposal on the subject of new board members ebay notes the board of directors does not endorse any icahn group nominee or the icahn proposal and unanimously recommends that you vote on the white proxy card or voting instruction form for all’ of the nominees proposed by the board of directors and against’ the icahn proposal” michael r jacobson company secretary writes in the sec filing the boar

### Readability

#### Utils

In [41]:
pip install textstat

     |████████████████████████████████| 101 kB 5.7 MB/s 
     |████████████████████████████████| 2.0 MB 9.2 MB/s 


In [42]:
import spacy
from textstat.textstat import textstatistics,legacy_round

def break_in_sentences(text):
	nlp = spacy.load('en_core_web_sm')
	doc = nlp(text)
	return list(doc.sents)
 
def number_of_sentences(text):
	sentences = break_in_sentences(text)
	return len(sentences)

def number_of_words(text):
	text_sentences = break_in_sentences(text)
	number_of_words = 0
	for sentence in text_sentences:
		number_of_words += len([token for token in sentence])
	return number_of_words
 
def number_of_syllables(word):
	return textstatistics().syllable_count(word)

def average_sentence_length(text):
	n_words = number_of_words(text)
	n_sentences = number_of_sentences(text)
	average_sentence_length = n_words/n_sentences
	return float(average_sentence_length)

def average_number_of_syllables_per_word(text):
  n_syllables = number_of_syllables(text)
  n_words = number_of_words(text)
  ASPW = float(n_syllables) / float(n_words)
  return legacy_round(ASPW, 1)

def number_of_difficult_words(text):
	nlp = spacy.load('en_core_web_sm')
	doc = nlp(text)

	words = []
	sentences = break_in_sentences(text)
	for sentence in sentences:
		words += [str(token) for token in sentence]

	diff_words_set = set()
	
	for word in words:
		syllable_count = number_of_syllables(word)
		if word not in nlp.Defaults.stop_words and syllable_count >= 2:
			diff_words_set.add(word)

	return len(diff_words_set)

def number_of_polysyllable_words(text):
	count = 0
	words = []
	sentences = break_in_sentences(text)
 
	for sentence in sentences:
		words += [token for token in sentence]

	for word in words:
		syllable_count = number_of_syllables(word)
		if syllable_count >= 3:
			count += 1
      
	return count

#### Flesch-Kincaid

In [43]:
def flesch_reading_index(text):
	FRE = 206.835 - float(1.015 * average_sentence_length(text)) - float(84.6 * average_number_of_syllables_per_word(text))
	return legacy_round(FRE, 2)

#### Dale-Chall

In [44]:
def dale_chall_index(text):  
  n_words = number_of_words(text)
  n_difficult_words = number_of_difficult_words(text)
  
  difficult_words_percentage = 0

  if n_words > 0:
		 difficult_words_percentage = float(n_difficult_words) / float(n_words) * 100
	
  DCI = (0.1579 * difficult_words_percentage) + (0.0496 * average_sentence_length(text)) + 3.6365
		
  return legacy_round(DCI, 2)

#### Gunning Fog

In [45]:
def gunning_fog(text):
    difficult_words_percentage = (number_of_difficult_words(text) / number_of_words(text)) * 100
    GFI = 0.4 * (average_sentence_length(text) + difficult_words_percentage)
    return GFI

#### Tests

In [46]:
text_to_test = df_news_aggregator.loc[1000]['Content']

In [47]:
flesch_reading_index(text_to_test)

91.57

In [48]:
dale_chall_index(text_to_test)

6.84

In [49]:
gunning_fog(text_to_test)

14.121428571428574

#### Dataset features

In [50]:
df_features_news_aggregator_names = ["Flesch-Kincaid", "Dale-Chall", "Bias"]
df_features_news_aggregator = pd.DataFrame(columns = df_features_news_aggregator_names)

In [51]:
for index, row in df_news_aggregator[:1].iterrows():
  text = row['Content']
  bias = row['Bias']
  flesch_kincaid = flesch_reading_index(text)
  dale_chall = dale_chall_index(text)
  df_features_news_aggregator = df_features_news_aggregator.append({'Flesch-Kincaid': flesch_kincaid, 'Dale-Chall': dale_chall, 'Bias': bias}, ignore_index=True)

In [52]:
df_features_news_aggregator

,Flesch-Kincaid,Dale-Chall,Bias
0,69.02,8.53,right-center


### n-grams

In [53]:
import re

def generate_ngrams(s, n):
    s = s.lower()
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [54]:
unigram = generate_ngrams(text_to_test, 1)
bigram = generate_ngrams(text_to_test, 2)

### TF-IDF

#### Utils

In [55]:
def compute_TF(word_dict, bag_of_words):
    tf_dict = {}
    bag_of_words_count = len(bag_of_words)
    for word, count in word_dict.items():
        tf_dict[word] = count / float(bag_of_words_count)
    return tf_dict

In [56]:
import math

def compute_IDF(documents):
    N = len(documents)
    
    idf_dict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idf_dict[word] += 1
    
    for word, val in idf_dict.items():
        idf_dict[word] = math.log(N / float(val))
    return idf_dict

In [57]:
def compute_TF_IDF(tf_bag_of_words, idfs):
    tf_idf = {}
    for word, val in tf_bag_of_words.items():
        tf_idf[word] = val * idfs[word]
    return tf_idf

#### Tests

In [64]:
corpus_news_aggregator_left = ""
corpus_news_aggregator_right = ""

for index, row in df_news_aggregator.iterrows():
    text = row['Content']
    if 'left' in row['Bias']:
        corpus_news_aggregator_left += preprocess(text)
    else: 
        corpus_news_aggregator_right += preprocess(text)

In [65]:
bag_of_words_news_aggregator_left = corpus_news_aggregator_left.split(' ')

In [66]:
bag_of_words_news_aggregator_right = corpus_news_aggregator_right.split(' ')

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

unique_words = set(bag_of_words_news_aggregator_left).union(set(bag_of_words_news_aggregator_right))

number_of_words_left = dict.fromkeys(unique_words, 0)
for word in bag_of_words_news_aggregator_left:
    number_of_words_left[word] += 1

number_of_words_right = dict.fromkeys(unique_words, 0)
for word in bag_of_words_news_aggregator_right:
    number_of_words_right[word] += 1

In [76]:
tf_left = compute_TF(number_of_words_left, bag_of_words_news_aggregator_left)
tf_right = compute_TF(number_of_words_right, bag_of_words_news_aggregator_right)